<a href="https://colab.research.google.com/github/Niranjana-08/AI-Ascent/blob/main/notebooks/analysis/sql_analysis_and_hypothesis_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook Objective :**


* The primary goal of this notebook is to perform an in-depth analysis of the cleaned and classified job data using SQL. We will connect to a SQLite database, run a series of analytical queries to uncover key trends, and then validate some of our core hypotheses with statistical tests (a T-test and a Chi-Square test).
* The final, aggregated dataframes from this analysis will be saved as CSV files for use in our final visualization dashboard.



## 1. Setup and Data Loading

In [1]:
import pandas as pd
from google.colab import drive
import sqlite3
import os
import shutil
from scipy import stats
from IPython.display import display

In [2]:
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

Mounting Google Drive...
Mounted at /content/drive


In [3]:
base_path = '/content/drive/My Drive/job-analysis/job-analysis-dataset/'
data_folder = base_path + 'data_cleaning/'

dashboard_file_path = data_folder + 'analysis_ready_without_combinedtext.csv'
modeling_file_path = data_folder + 'analysis_ready_with_combinedtext.csv'

In [4]:
print("\nLoading datasets")
try:
    #Primary DataFrame we will use for the SQL analysis
    dashboard_df = pd.read_csv(dashboard_file_path)

    #contains combined_text column extra
    modeling_df = pd.read_csv(modeling_file_path)

    print("Both datasets loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: A file was not found. Please check your folder and file names.")
    print(f"Details: {e}")
    raise e


Loading datasets...
Both datasets loaded successfully!


In [5]:
print("\n--- Verification ---")
print("1. dashboard_df (for SQL and visualization):")
display(dashboard_df.head(2))

print("\n2. modeling_df (for future modeling):")
display(modeling_df.head(2))


--- Verification ---
1. dashboard_df (for SQL and visualization):


,job_id,title,company_name,location,main_category,sub_category,ai_role_type,ai_relevance_score,formatted_experience_level,min_salary,med_salary,max_salary,pay_period,cleaned_skills,state_code
0,921716,Marketing Coordinator,Corcoran Sawyer Smith,"Princeton, NJ",Human Resources,Talent Acquisition & Recruiting,Traditional Role,0.260290,Not Specified,17.0,0.0,20.0,HOURLY,"Marketing, Sales",NJ
1,1829192,Mental Health Therapist/Counselor,Unknown,"Fort Collins, CO",Healthcare (Research & Admin),Clinical & Patient Care,Traditional Role,0.180139,Not Specified,30.0,0.0,50.0,HOURLY,Health Care Provider,CO



2. modeling_df (for future modeling):


,job_id,title,company_name,location,main_category,sub_category,ai_role_type,ai_relevance_score,formatted_experience_level,min_salary,med_salary,max_salary,pay_period,cleaned_skills,combined_text,state_code
0,921716,Marketing Coordinator,Corcoran Sawyer Smith,"Princeton, NJ",Human Resources,Talent Acquisition & Recruiting,Traditional Role,0.260290,Not Specified,17.0,0.0,20.0,HOURLY,"Marketing, Sales",marketing coordinator marketing sale job descr...,NJ
1,1829192,Mental Health Therapist/Counselor,Unknown,"Fort Collins, CO",Healthcare (Research & Admin),Clinical & Patient Care,Traditional Role,0.180139,Not Specified,30.0,0.0,50.0,HOURLY,Health Care Provider,mental health therapist counselor health care ...,CO


## 2. Database Creation

In [7]:
# Defining a LOCAL path and the final Google Drive path : Creating a database
local_db_path = 'jobs_database.db'
# final files are stored to :
drive_folder_path = '/content/drive/My Drive/job-analysis-dataset/analysis/'
drive_db_path = os.path.join(drive_folder_path, 'jobs_database.db')

In [8]:
os.makedirs(drive_folder_path, exist_ok=True)
print(f"Ensured destination folder exists: {drive_folder_path}")

Ensured destination folder exists: /content/drive/My Drive/job-analysis-dataset/analysis/


In [9]:
print(f"\nCreating database locally at: {local_db_path}")
conn = sqlite3.connect(local_db_path)
dashboard_df.to_sql('jobs', conn, if_exists='replace', index=False)
conn.close()
print("DataFrame successfully loaded into the local database.")


Creating database locally at: jobs_database.db
DataFrame successfully loaded into the local database.


In [10]:
print(f"\nCopying database to Google Drive at: {drive_db_path}")
shutil.copy(local_db_path, drive_db_path)
print("Database file successfully copied to Google Drive!")


Copying database to Google Drive at: /content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db
Database file successfully copied to Google Drive!


## 3. SQL Analysis: Uncovering Job Market Insights

Now we will connect to the database in Google Drive and run a series of SQL queries to answer specific questions about the job market.

### Analysis 1: High-Level Job Category Breakdown

*Question: What is the overall distribution of jobs across different industries, and how does AI influence each?*

In [11]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [12]:
sql_query = """
SELECT
    main_category,
    COUNT(job_id) AS total_jobs,

    -- Calculate the percentage of total jobs for each category
    CAST(COUNT(job_id) AS REAL) * 100 / (SELECT COUNT(*) FROM jobs WHERE main_category != 'Other') AS percentage_of_total,

    -- Calculate the average AI relevance score for each category
    AVG(ai_relevance_score) AS average_ai_score,

    -- Count the number of 'Core AI Roles' in each category
    SUM(CASE WHEN ai_role_type = 'Core AI Role' THEN 1 ELSE 0 END) AS core_ai_jobs,

    -- Count the number of 'AI-Impacted Roles' in each category
    SUM(CASE WHEN ai_role_type = 'AI-Impacted Role' THEN 1 ELSE 0 END) AS ai_impacted_jobs,

    -- Count the number of 'Traditional Roles' in each category
    SUM(CASE WHEN ai_role_type = 'Traditional Role' THEN 1 ELSE 0 END) AS traditional_jobs

FROM
    jobs
WHERE
    main_category != 'Other' -- Exclude the 'Other' category from this analysis
GROUP BY
    main_category
ORDER BY
    total_jobs DESC;
"""

In [13]:
print("\nRunning analysis query...")
category_analysis_df = pd.read_sql_query(sql_query, conn)
print("Analysis complete.")


Running analysis query...
Analysis complete.


In [14]:
conn.close()
print("\nConnection closed.")


Connection closed.


In [15]:
print("\n--- High-Value Insights per Job Category ---")
pd.set_option('display.float_format', '{:.2f}'.format)
display(category_analysis_df)


--- High-Value Insights per Job Category ---


,main_category,total_jobs,percentage_of_total,average_ai_score,core_ai_jobs,ai_impacted_jobs,traditional_jobs
0,Technology,17734,20.36,0.31,93,17641,0
1,Consulting & Strategy,15641,17.96,0.27,731,3563,11347
2,Healthcare (Research & Admin),15174,17.42,0.24,792,2510,11872
3,Marketing,9019,10.35,0.27,418,1813,6788
4,Human Resources,6708,7.70,0.24,297,1412,4999
5,Finance,6637,7.62,0.24,307,1216,5114
6,Supply Chain & Logistics,6031,6.92,0.25,311,1368,4352
7,Automotive,5091,5.84,0.28,251,904,3936
8,Education & EdTech,2024,2.32,0.20,94,448,1482
9,Legal,1697,1.95,0.21,100,309,1288


### Analysis 2: Experience Level by AI Role Type

*Question: How does the required experience level differ between Traditional, AI-Impacted, and Core AI roles?*

In [16]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [17]:
sql_query_experience = """
SELECT
    ai_role_type,
    formatted_experience_level,
    COUNT(job_id) AS total_jobs
FROM
    jobs
WHERE
    main_category != 'Other' AND
    formatted_experience_level != 'Not Specified' -- Exclude 'Not Specified' for a cleaner analysis
GROUP BY
    ai_role_type,
    formatted_experience_level
ORDER BY
    ai_role_type,
    total_jobs DESC;
"""

In [18]:
print("\nRunning experience level analysis query...")
experience_analysis_df = pd.read_sql_query(sql_query_experience, conn)
print("Analysis complete.")


Running experience level analysis query...
Analysis complete.


In [19]:
conn.close()

print("\n--- Insights: Experience Level by AI Role Type ---")
display(experience_analysis_df)


--- Insights: Experience Level by AI Role Type ---


,ai_role_type,formatted_experience_level,total_jobs
0,AI-Impacted Role,Mid-Senior level,11642
1,AI-Impacted Role,Entry level,7619
2,AI-Impacted Role,Associate,2640
3,AI-Impacted Role,Director,618
4,AI-Impacted Role,Internship,415
5,AI-Impacted Role,Executive,169
6,Core AI Role,Mid-Senior level,1029
7,Core AI Role,Entry level,871
8,Core AI Role,Associate,373
9,Core AI Role,Director,68


### Analysis 3: Salary Comparison by AI Role and Experience

*Question: Is there a salary premium for AI-related roles at different experience levels?*

In [20]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [21]:
sql_query_salary = """
SELECT
    ai_role_type,
    formatted_experience_level,
    -- Calculate the average MEDIAN salary for accuracy, and cast to integer
    CAST(AVG(med_salary) AS INTEGER) AS average_median_salary,
    -- Count the total jobs in this segment
    COUNT(job_id) AS total_jobs,
    -- Count the number of unique companies hiring for this segment
    COUNT(DISTINCT company_name) AS unique_companies_hiring
FROM
    jobs
WHERE
    -- We only include jobs that have salary data and a specified experience level
    med_salary > 0 AND
    formatted_experience_level != 'Not Specified'
GROUP BY
    ai_role_type,
    formatted_experience_level
ORDER BY
    -- Order by role type and then by salary to see the progression
    ai_role_type,
    average_median_salary DESC;
"""

In [22]:
print("\nRunning salary analysis query...")
salary_analysis_df = pd.read_sql_query(sql_query_salary, conn)
print("Analysis complete.")


Running salary analysis query...
Analysis complete.


In [23]:
conn.close()

# calculating no of jobs with salary data and getting the count of all jobs that have a salary > 0
conn = sqlite3.connect(db_path)
jobs_with_salary_count = pd.read_sql_query("SELECT COUNT(*) FROM jobs WHERE med_salary > 0;", conn).iloc[0,0]
total_job_count = pd.read_sql_query("SELECT COUNT(*) FROM jobs;", conn).iloc[0,0]
conn.close()

# Printing the req disclaimer that the user should know
print(f"\nNote: The salary analysis above is based on {jobs_with_salary_count} jobs "
      f"out of a total of {total_job_count} that provided salary information.")

print("\n--- Insights: Salary by Experience and AI Role Type ---")
display(salary_analysis_df)


Note: The salary analysis above is based on 6266 jobs out of a total of 123849 that provided salary information.

--- Insights: Salary by Experience and AI Role Type ---


,ai_role_type,formatted_experience_level,average_median_salary,total_jobs,unique_companies_hiring
0,AI-Impacted Role,Executive,192174,6,5
1,AI-Impacted Role,Director,138470,7,6
2,AI-Impacted Role,Mid-Senior level,43297,288,178
3,AI-Impacted Role,Associate,20477,111,78
4,AI-Impacted Role,Entry level,13188,473,219
5,AI-Impacted Role,Internship,2798,29,26
6,Core AI Role,Director,212000,3,3
7,Core AI Role,Mid-Senior level,27526,16,15
8,Core AI Role,Associate,13958,19,17
9,Core AI Role,Entry level,9659,55,37


### Analysis 4: Salary Range Comparison by AI Role and Experience

Question: How do the full salary ranges (minimum to maximum) compare across the different AI role tiers and experience levels? This helps us understand not just the average pay, but the potential for earnings growth.

In [24]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [25]:
jobs_with_salary_count = pd.read_sql_query("SELECT COUNT(*) FROM jobs WHERE min_salary > 0;", conn).iloc[0,0]
total_job_count = pd.read_sql_query("SELECT COUNT(*) FROM jobs;", conn).iloc[0,0]

In [26]:
sql_query_salary_range = """
SELECT
    ai_role_type,
    formatted_experience_level,
    CAST(AVG(min_salary) AS INTEGER) AS average_minimum_salary,
    CAST(AVG(max_salary) AS INTEGER) AS average_maximum_salary,
    COUNT(job_id) AS total_jobs
FROM
    jobs
WHERE
    min_salary > 0 AND
    formatted_experience_level != 'Not Specified'
GROUP BY
    ai_role_type,
    formatted_experience_level
ORDER BY
    ai_role_type,
    average_minimum_salary DESC;
"""


In [27]:
print("\nRunning salary range analysis query...")
salary_range_df = pd.read_sql_query(sql_query_salary_range, conn)
print("Analysis complete.")


Running salary range analysis query...
Analysis complete.


In [28]:
conn.close()

print("\n--- Insights: Salary Range by Experience and AI Role Type ---")
print(f"\nNote: The salary analysis below is based on {jobs_with_salary_count} jobs "
      f"out of a total of {total_job_count} that provided salary information.")
display(salary_range_df)


--- Insights: Salary Range by Experience and AI Role Type ---

Note: The salary analysis below is based on 29793 jobs out of a total of 123849 that provided salary information.


,ai_role_type,formatted_experience_level,average_minimum_salary,average_maximum_salary,total_jobs
0,AI-Impacted Role,Executive,193666,265498,46
1,AI-Impacted Role,Director,141752,201144,202
2,AI-Impacted Role,Mid-Senior level,76393,107420,3596
3,AI-Impacted Role,Entry level,47438,69250,1600
4,AI-Impacted Role,Associate,43567,57203,1031
5,AI-Impacted Role,Internship,8809,14787,80
6,Core AI Role,Director,162786,231228,30
7,Core AI Role,Executive,101662,121294,8
8,Core AI Role,Mid-Senior level,69357,98079,309
9,Core AI Role,Entry level,41240,61718,161


In [29]:
'''insights for the below graph :
Here are a few key takeaways from this table that show the analysis is working well:
Clear Seniority Progression: For every ai_role_type (Traditional, AI-Impacted), the salaries increase logically with the formatted_experience_level, from Internship up to Executive. This is a strong sign that the data is sound.
Significant "AI Premium": If you compare roles at the same experience level, AI-Impacted Roles consistently have higher average salaries than Traditional Roles.
Mid-Senior level: $81,566 (AI-Impacted) vs. $69,155 (Traditional)
Entry level: $56,825 (AI-Impacted) vs. $27,222 (Traditional) – more than double!
"Core AI" Pays the Most: The few "Core AI" roles shown have the highest salaries of all, which is what we would expect.
'''

'insights for the below graph :\nHere are a few key takeaways from this table that show the analysis is working well:\nClear Seniority Progression: For every ai_role_type (Traditional, AI-Impacted), the salaries increase logically with the formatted_experience_level, from Internship up to Executive. This is a strong sign that the data is sound.\nSignificant "AI Premium": If you compare roles at the same experience level, AI-Impacted Roles consistently have higher average salaries than Traditional Roles.\nMid-Senior level: $81,566 (AI-Impacted) vs. $69,155 (Traditional)\nEntry level: $56,825 (AI-Impacted) vs. $27,222 (Traditional) – more than double!\n"Core AI" Pays the Most: The few "Core AI" roles shown have the highest salaries of all, which is what we would expect.\n'

### Analysis 5: Top 20 AI Job Hubs

*Question: Geographically, where is the demand for AI-related jobs most concentrated?*

In [30]:
import pandas as pd
import sqlite3
from IPython.display import display

In [31]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [32]:
# Counting only the AI-related jobs
ai_job_count = pd.read_sql_query("SELECT COUNT(*) FROM jobs WHERE ai_role_type IN ('Core AI Role', 'AI-Impacted Role');", conn).iloc[0,0]
total_job_count = pd.read_sql_query("SELECT COUNT(*) FROM jobs;", conn).iloc[0,0]

In [33]:
sql_query_location = """
SELECT
    location,
    COUNT(job_id) AS total_ai_jobs
FROM
    jobs
WHERE
    ai_role_type IN ('Core AI Role', 'AI-Impacted Role') AND
    location != 'Not Specified'
GROUP BY
    location
ORDER BY
    total_ai_jobs DESC
LIMIT 20; -- We'll look at the top 20 locations
"""

In [34]:
print("\nRunning location analysis query...")
location_df = pd.read_sql_query(sql_query_location, conn)
print("Analysis complete.")


Running location analysis query...
Analysis complete.


In [35]:
conn.close()

print("\n--- Insights: Top 20 AI Job Hubs by Volume ---")
display(location_df)

print(f"\nNote: The analysis above considers only the {ai_job_count} 'Core AI' and 'AI-Impacted' jobs "
      f"out of the {total_job_count} total jobs in the dataset.")


--- Insights: Top 20 AI Job Hubs by Volume ---


,location,total_ai_jobs
0,United States,3846
1,"New York, NY",718
2,"Chicago, IL",560
3,"Atlanta, GA",536
4,"Houston, TX",529
5,"Dallas, TX",446
6,"Austin, TX",419
7,"Boston, MA",376
8,"Phoenix, AZ",370
9,"Seattle, WA",330



Note: The analysis above considers only the 34881 'Core AI' and 'AI-Impacted' jobs out of the 123849 total jobs in the dataset.


### Analysis 6: Experience Level by Industry

*Question: What is the seniority breakdown of jobs within each industry? This helps identify which sectors have more entry-level opportunities versus those that are more senior-heavy.*



In [36]:
import pandas as pd
import sqlite3
from IPython.display import display

In [37]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [38]:
sql_query_experience_industry = """
SELECT
    main_category,
    formatted_experience_level,
    COUNT(job_id) AS job_count
FROM
    jobs
WHERE
    main_category != 'Other' AND
    formatted_experience_level != 'Not Specified'
GROUP BY
    main_category,
    formatted_experience_level
ORDER BY
    main_category,
    job_count DESC;
"""

In [39]:
print("\nRunning experience level by industry analysis query...")
experience_industry_df = pd.read_sql_query(sql_query_experience_industry, conn)
print("Analysis complete.")


Running experience level by industry analysis query...
Analysis complete.


In [40]:
conn.close()

pd.set_option('display.max_rows', None)
display(experience_industry_df)

# --- Optional: Reset the option back to default after viewing ---
# pd.reset_option('display.max_rows')

,main_category,formatted_experience_level,job_count
0,Automotive,Entry level,2396
1,Automotive,Mid-Senior level,1152
2,Automotive,Associate,357
3,Automotive,Internship,197
4,Automotive,Director,27
5,Automotive,Executive,8
6,Consulting & Strategy,Mid-Senior level,6467
7,Consulting & Strategy,Entry level,2225
8,Consulting & Strategy,Associate,1769
9,Consulting & Strategy,Director,856


In [41]:
entry_level_df = experience_industry_df[experience_industry_df['formatted_experience_level'] == 'Entry level'].sort_values('job_count', ascending=False)
top_entry_level_cat = entry_level_df.iloc[0]['main_category']
top_entry_level_count = entry_level_df.iloc[0]['job_count']

senior_level_df = experience_industry_df[experience_industry_df['formatted_experience_level'] == 'Mid-Senior level'].sort_values('job_count', ascending=False)
top_senior_level_cat = senior_level_df.iloc[0]['main_category']
top_senior_level_count = senior_level_df.iloc[0]['job_count']

In [42]:
print("\n" + "="*50)
print("Why We Created This Table")
print("="*50)
print("\nWe created this table to understand the seniority breakdown of jobs within each industry. "
      "It helps us answer key questions for job seekers planning their careers.")

print(f"\nInsight 1: Where are the most entry-level opportunities?")
print(f"The data shows that the '{top_entry_level_cat}' category offers the most entry-level positions, with {top_entry_level_count} jobs listed in our dataset.")

print(f"\nInsight 2: Where are the most senior roles concentrated?")
print(f"The '{top_senior_level_cat}' category has the highest concentration of 'Mid-Senior level' roles, with {top_senior_level_count} jobs.")


Why We Created This Table

We created this table to understand the seniority breakdown of jobs within each industry. It helps us answer key questions for job seekers planning their careers.

Insight 1: Where are the most entry-level opportunities?
The data shows that the 'Healthcare (Research & Admin)' category offers the most entry-level positions, with 4980 jobs listed in our dataset.

Insight 2: Where are the most senior roles concentrated?
The 'Technology' category has the highest concentration of 'Mid-Senior level' roles, with 7292 jobs.


###  Analysis 7: In-Demand Skills for AI Jobs

*Question: What are the most frequently mentioned skills for AI-related roles, both overall and within each specific industry?*

In [43]:
import pandas as pd
import sqlite3
from IPython.display import display

In [44]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


TYPE 1 :

In [45]:
print("\n--- Running Analysis 1: Overall Top In-Demand Skills ---")
sql_query_overall_skills = """
WITH RECURSIVE split(job_id, skill, rest) AS (
    SELECT
        job_id,
        '',
        cleaned_skills || ','
    FROM
        jobs
    WHERE
        ai_role_type IN ('Core AI Role', 'AI-Impacted Role') AND
        cleaned_skills != ''

    UNION ALL

    SELECT
        job_id,
        TRIM(SUBSTR(rest, 1, INSTR(rest, ',') - 1)),
        SUBSTR(rest, INSTR(rest, ',') + 1)
    FROM
        split
    WHERE
        rest != ''
)
SELECT
    skill,
    COUNT(job_id) AS job_count
FROM
    split
WHERE
    skill != ''
GROUP BY
    skill
ORDER BY
    job_count DESC
LIMIT 15;
"""
overall_skills_df = pd.read_sql_query(sql_query_overall_skills, conn)
print("Analysis 1 complete.")


--- Running Analysis 1: Overall Top In-Demand Skills ---
Analysis 1 complete.


In [46]:
print("\n--- Results: Overall Top 15 In-Demand Skills for AI Jobs ---")
display(overall_skills_df)


--- Results: Overall Top 15 In-Demand Skills for AI Jobs ---


,skill,job_count
0,Information Technology,15580
1,Engineering,8862
2,Management,3801
3,Manufacturing,3288
4,Other,2505
5,Sales,2488
6,Health Care Provider,2309
7,Analyst,1773
8,Business Development,1635
9,Finance,1529


TYPE 2 :

In [47]:
print("\n--- Running Analysis 2: Top 5 In-Demand Skills per Category ---")

# Counting and taking into consideration only the jobs wiht skills
ai_job_count_with_skills = pd.read_sql_query("SELECT COUNT(*) FROM jobs WHERE ai_role_type IN ('Core AI Role', 'AI-Impacted Role') AND cleaned_skills != '';", conn).iloc[0,0]
sql_query_skills_per_category = """
WITH RECURSIVE split(job_id, main_category, skill, rest) AS (
    SELECT
        job_id,
        main_category,
        '',
        cleaned_skills || ','
    FROM
        jobs
    WHERE
        ai_role_type IN ('Core AI Role', 'AI-Impacted Role') AND
        cleaned_skills != '' AND
        main_category != 'Other'

    UNION ALL

    SELECT
        job_id,
        main_category,
        TRIM(SUBSTR(rest, 1, INSTR(rest, ',') - 1)),
        SUBSTR(rest, INSTR(rest, ',') + 1)
    FROM
        split
    WHERE
        rest != ''
),
CategorySkillCounts AS (
    SELECT
        main_category,
        skill,
        COUNT(job_id) AS job_count,
        RANK() OVER(PARTITION BY main_category ORDER BY COUNT(job_id) DESC) as rank_num
    FROM
        split
    WHERE
        skill != ''
    GROUP BY
        main_category,
        skill
)
SELECT
    main_category,
    skill,
    job_count
FROM
    CategorySkillCounts
WHERE
    rank_num <= 5
-- Corrected ORDER BY clause to sort by count first, then category name
ORDER BY
    job_count DESC,
    main_category;
"""
skills_per_category_df = pd.read_sql_query(sql_query_skills_per_category, conn)
print("Analysis 2 complete.")
print("\n--- Results: Top 5 In-Demand Skills per Main Category ---")
display(skills_per_category_df)


print(f"\nNote: The analysis above considers only the {ai_job_count_with_skills} 'Core AI' and 'AI-Impacted' jobs "
      f"that had specific skills listed.")


--- Running Analysis 2: Top 5 In-Demand Skills per Category ---
Analysis 2 complete.

--- Results: Top 5 In-Demand Skills per Main Category ---


,main_category,skill,job_count
0,Technology,Information Technology,11400
1,Technology,Engineering,7364
2,Healthcare (Research & Admin),Health Care Provider,2145
3,Consulting & Strategy,Information Technology,1974
4,Technology,Management,1550
5,Technology,Manufacturing,1297
6,Technology,Other,1249
7,Supply Chain & Logistics,Manufacturing,955
8,Supply Chain & Logistics,Management,896
9,Finance,Finance,869



Note: The analysis above considers only the 34201 'Core AI' and 'AI-Impacted' jobs that had specific skills listed.


In [48]:
conn.close()

### Analysis 8: In-Demand Job Roles (Sub-Category)

*Question: Which specific job roles are most common, both overall and within each industry?*

TYPE 1 : to find the overall top 10 most in-demand roles

In [49]:
import pandas as pd
import sqlite3
from IPython.display import display

In [50]:
db_path = '/content/drive/My Drive/job-analysis-dataset/analysis/jobs_database.db'
conn = sqlite3.connect(db_path)
print("Successfully re-connected to the database.")

Successfully re-connected to the database.


In [51]:
print("\n--- Running Analysis 1: Overall Top 10 Most In-Demand Roles ---")
sql_query_overall_sub = """
SELECT
    sub_category,
    COUNT(job_id) AS job_count
FROM
    jobs
WHERE
    sub_category NOT IN ('Other', 'Not Found')
GROUP BY
    sub_category
ORDER BY
    job_count DESC
LIMIT 10;
"""
overall_sub_df = pd.read_sql_query(sql_query_overall_sub, conn)
print("Analysis 1 complete.")


--- Running Analysis 1: Overall Top 10 Most In-Demand Roles ---
Analysis 1 complete.


In [52]:
print("\n--- Results: Overall Top 10 Most In-Demand Job Roles ---")
display(overall_sub_df)


--- Results: Overall Top 10 Most In-Demand Job Roles ---


,sub_category,job_count
0,Specialized & Domain-Specific Advisory,13965
1,Clinical & Patient Care,13077
2,"Infrastructure, Cloud & Operations",6400
3,Marketing Strategy & Analytics,6381
4,"Cybersecurity, QA & Specialized Tech",5605
5,Logistics & Operations,5318
6,Core Engineering & Systems,4933
7,Core HR & Business Partnership,3907
8,"Accounting, Assurance & Tax",3741
9,"Data, AI & Analytics",3124


TYPE 2 : to find the top 3 roles within each main industry

In [53]:
print("\n--- Running Analysis 2: Top 3 Roles Within Each Industry ---")
sql_query_sub_per_main = """
WITH RankedSubCategories AS (
    SELECT
        main_category,
        sub_category,
        COUNT(job_id) AS job_count,
        RANK() OVER(PARTITION BY main_category ORDER BY COUNT(job_id) DESC) as rank_num
    FROM
        jobs
    WHERE
        main_category != 'Other' AND
        sub_category NOT IN ('Other', 'Not Found')
    GROUP BY
        main_category,
        sub_category
)
SELECT
    main_category,
    sub_category,
    job_count
FROM
    RankedSubCategories
WHERE
    rank_num <= 3
ORDER BY
    main_category,
    job_count DESC;
"""
sub_per_main_df = pd.read_sql_query(sql_query_sub_per_main, conn)
print("Analysis 2 complete.")


--- Running Analysis 2: Top 3 Roles Within Each Industry ---
Analysis 2 complete.


In [54]:
print("\n--- Results: Top 3 Most In-Demand Roles Within Each Industry ---")
display(sub_per_main_df)


--- Results: Top 3 Most In-Demand Roles Within Each Industry ---


,main_category,sub_category,job_count
0,Automotive,Core Engineering & Systems,4933
1,Automotive,Electric & Autonomous Vehicles (EV/AV),133
2,Automotive,Software & Electronics,25
3,Consulting & Strategy,Specialized & Domain-Specific Advisory,13965
4,Consulting & Strategy,Core & Management Strategy,1478
5,Consulting & Strategy,Major Consulting Firms,198
6,Design,Specialized Design Roles,838
7,Design,Digital Product & UX/UI,400
8,Design,Visual & Graphic Design,22
9,Education & EdTech,Academic & Training Roles,1501


In [55]:
conn.close()

## 4. Hypothesis Testing

To add statistical rigor to our findings, we will perform two hypothesis tests.

In [56]:
import pandas as pd
from scipy import stats
from IPython.display import display

In [57]:
print("Preparing salary data for the two groups from dashboard_df...")
salary_df = dashboard_df[dashboard_df['med_salary'] > 0].copy()

Preparing salary data for the two groups from dashboard_df...


In [58]:
ai_impacted_salaries = salary_df[salary_df['ai_role_type'] == 'AI-Impacted Role']['med_salary']
traditional_salaries = salary_df[salary_df['ai_role_type'] == 'Traditional Role']['med_salary']

print(f"Found {len(ai_impacted_salaries)} AI-Impacted jobs with salary data.")
print(f"Found {len(traditional_salaries)} Traditional jobs with salary data.")

Found 1118 AI-Impacted jobs with salary data.
Found 5027 Traditional jobs with salary data.


### Test 1: T-test for Salary Difference

*Hypothesis: Is the observed difference in median salary between "AI-Impacted" and "Traditional" roles statistically significant, or could it be due to random chance?*

In [59]:
print("\nPerforming t-test...")
t_stat, p_value = stats.ttest_ind(ai_impacted_salaries, traditional_salaries, equal_var=False, nan_policy='omit')
print("T-test complete.")


Performing t-test...
T-test complete.


In [60]:
print("\n" + "="*50)
print("Hypothesis Test Results")
print("="*50)

print("\nOur question: Is the difference in salary between 'AI-Impacted' and 'Traditional' roles real, or just random chance?")
print(f"\nThe p-value is: {p_value:.10f}")


Hypothesis Test Results

Our question: Is the difference in salary between 'AI-Impacted' and 'Traditional' roles real, or just random chance?

The p-value is: 0.0415101072


In [61]:
alpha = 0.05
if p_value < alpha:
    print("\nConclusion: YES, the result is statistically significant.")
    print("Because the p-value is very small (less than 0.05), we can confidently say that the difference in salaries is real.")
else:
    print("\nConclusion: NO, the result is not statistically significant.")
    print("Because the p-value is high (greater than 0.05), we cannot say the salary difference is real; it could be due to random chance.")


print("\n--- Average Salary Comparison ---")
summary_table = salary_df.groupby('ai_role_type')['med_salary'].agg(['mean', 'count'])
summary_table['mean'] = summary_table['mean'].map('${:,.2f}'.format)
display(summary_table)


Conclusion: YES, the result is statistically significant.
Because the p-value is very small (less than 0.05), we can confidently say that the difference in salaries is real.

--- Average Salary Comparison ---


,mean,count
ai_role_type,,
AI-Impacted Role,"$25,114.47",1118
Core AI Role,"$17,138.08",121
Traditional Role,"$21,505.15",5027


### Test 2: Chi-Square Test for Industry and AI Role Relationship

*Hypothesis: Is there a statistically significant relationship between the industry (main_category) of a job and its ai_role_type?*

In [63]:
print("Creating a contingency table (cross-tabulation)...")
contingency_table = pd.crosstab(
    dashboard_df['main_category'],
    dashboard_df['ai_role_type']
)
if 'Other' in contingency_table.index:
    contingency_table = contingency_table.drop('Other')

print("Contingency Table:")
display(contingency_table)

Creating a contingency table (cross-tabulation)...
Contingency Table:


ai_role_type,AI-Impacted Role,Core AI Role,Traditional Role
main_category,,,
Automotive,904,251,3936
Consulting & Strategy,3563,731,11347
Design,224,51,985
Education & EdTech,448,94,1482
Finance,1216,307,5114
Healthcare (Research & Admin),2510,792,11872
Human Resources,1412,297,4999
Legal,309,100,1288
Marketing,1813,418,6788


In [64]:
print("\nPerforming Chi-Square test...")
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)
print("Test complete.")


Performing Chi-Square test...
Test complete.


In [65]:
print("\n" + "="*50)
print("Chi-Square Test Results")
print("="*50)

print("\nOur question: Is there a real relationship between industry and the adoption of AI roles?")
print(f"\nThe p-value is: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("\nConclusion: YES, the result is statistically significant.")
    print("Because the p-value is extremely small, we can confidently say there is a significant relationship between a job's category and its AI role type.")
else:
    print("\nConclusion: NO, the result is not statistically significant.")
    print("We cannot say there is a real relationship between a job's category and its AI role type.")



Chi-Square Test Results

Our question: Is there a real relationship between industry and the adoption of AI roles?

The p-value is: 0.0

Conclusion: YES, the result is statistically significant.
Because the p-value is extremely small, we can confidently say there is a significant relationship between a job's category and its AI role type.


## 5. Save Analysis Files

Finally, we'll save all the aggregated dataframes created during our SQL analysis to CSV files. These will serve as the data sources for our visualization notebook.

In [66]:
# Main destination folder
analysis_folder_path = '/content/drive/My Drive/job-analysis/job-analysis-dataset/analysis/'
print(f"Saving analysis files to: {analysis_folder_path}")

if 'category_analysis_df' in locals():
    category_analysis_df.to_csv(analysis_folder_path + 'summary_by_main_category.csv', index=False)
    print("Saved summary_by_main_category.csv")

if 'experience_analysis_df' in locals():
    experience_analysis_df.to_csv(analysis_folder_path + 'summary_by_experience.csv', index=False)
    print("Saved summary_by_experience.csv")

if 'salary_analysis_df' in locals():
    salary_analysis_df.to_csv(analysis_folder_path + 'summary_salary_median.csv', index=False)
    print("Saved summary_salary_median.csv")

if 'salary_range_df' in locals():
    salary_range_df.to_csv(analysis_folder_path + 'summary_salary_range.csv', index=False)
    print("Saved summary_salary_range.csv")

if 'location_df' in locals():
    location_df.to_csv(analysis_folder_path + 'summary_top_locations.csv', index=False)
    print("Saved summary_top_locations.csv")

if 'overall_skills_df' in locals():
    overall_skills_df.to_csv(analysis_folder_path + 'summary_overall_top_skills.csv', index=False)
    print("Saved summary_overall_top_skills.csv")

if 'skills_per_category_df' in locals():
    skills_per_category_df.to_csv(analysis_folder_path + 'summary_skills_per_category.csv', index=False)
    print("Saved summary_skills_per_category.csv")

print("\nAll analysis files have been saved and are ready for your visualization notebook.")

Saving analysis files to: /content/drive/My Drive/job-analysis/job-analysis-dataset/analysis/
Saved summary_by_main_category.csv
Saved summary_by_experience.csv
Saved summary_salary_median.csv
Saved summary_salary_range.csv
Saved summary_top_locations.csv
Saved summary_overall_top_skills.csv
Saved summary_skills_per_category.csv

All analysis files have been saved and are ready for your visualization notebook.


### End of analysis